# RAG Sandbox

In [2]:
import os

from langfuse.llama_index import LlamaIndexCallbackHandler
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

In [3]:
langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key="pk-lf-363e8a50-c2ef-4622-b47b-0fd9db3b90eb",
    secret_key=os.environ["LANGFUSE_SECRET_KEY"],
    host="http://localhost:3000",
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

In [4]:
MODEL = "llama3.2:1b"
# MODEL = "deepseek-r1:1.5b"
# MODEL = "gemma2:2b"
# MODEL = "llama3.2:3b"
# MODEL = "mistral:7b"
# MODEL = "deepseek-r1:7b"
# MODEL = "llama3.1:8b"
# MODEL = "gemma2:9b"
# MODEL = "phi4:14b"
# MODEL = "deepseek-r1:14b"
# MODEL = "llama3.3:70b"

EMBEDDER = "all-minilm"  # 384
# EMBEDDER = "nomic-embed-text" # 768
# EMBEDDER = "bge-m3" # 1024
# EMBEDDER = MODEL

llm = Ollama(model=MODEL, request_timeout=60.0)
# Settings.llm = llm
Settings.embed_model = OllamaEmbedding(model_name=EMBEDDER)

In [ ]:
response = llm.complete("What is the capital of France?")
print(response)

In [ ]:
response = llm.stream_complete("Tell me a short story about a robot.")
for token in response:
    print(token.delta, end="", flush=True)